# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,110.906033,0.011457,1234,110.917490,110.894576,221.834981,221.789153
6,231.026123,-0.626592,1234,230.399532,231.652715,460.799063,463.305429
7,75.673626,-0.441323,1234,75.232302,76.114949,150.464605,152.229897
8,198.825809,-0.776552,1234,198.049257,199.602361,396.098514,399.204721
9,860.818171,0.849109,1234,861.667280,859.969061,1723.334560,1719.938123
10,871.501180,0.850785,1234,872.351964,870.650395,1744.703928,1741.300790
11,435.293759,0.807015,1234,436.100774,434.486744,872.201548,868.973488
12,428.138487,-0.880397,1234,427.258090,429.018884,854.516180,858.037769
13,588.779540,0.452320,1234,589.231860,588.327220,1178.463720,1176.654441
14,941.239178,0.657648,1234,941.896826,940.581529,1883.793651,1881.163059


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-110.917490,110.894576
6,-230.399532,231.652715
7,-75.232302,76.114949
8,-198.049257,199.602361
9,-861.667280,859.969061
10,-872.351964,870.650395
11,-436.100774,434.486744
12,-427.258090,429.018884
13,-589.231860,588.327220
14,-941.896826,940.581529


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-110.917490,110.894576
6,-230.399532,231.652715
7,-75.232302,76.114949
8,-198.049257,199.602361
9,-861.667280,859.969061
10,-872.351964,870.650395
11,-436.100774,434.486744
12,-427.258090,429.018884
13,-589.231860,588.327220
14,-941.896826,940.581529


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-110.91749029,  110.89457634],
       [-230.39953166,  231.65271472],
       [ -75.23230234,   76.1149487 ],
       [-198.04925688,  199.60236064],
       [-861.66728017,  859.96906134],
       [-872.35196415,  870.65039512],
       [-436.10077381,  434.48674408],
       [-427.25808998,  429.01888442],
       [-589.23186011,  588.3272204 ],
       [-941.89682573,  940.58152947]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,219.834981,-221.789153
6,458.799063,-463.305429
7,148.464605,-152.229897
8,394.098514,-399.204721
9,1721.334560,-1719.938123
10,1742.703928,-1741.300790
11,870.201548,-868.973488
12,852.516180,-858.037769
13,1176.463720,-1176.654441
14,1881.793651,-1881.163059


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
